In [1]:
import sys
sys.path.append('../')
from utils import *
from model import *

In [ ]:
data_dir = '/datadrive/data_cs/'
batch_size = 256
batch_num  = 128
epoch_num  = 100
samp_num   = 7

device = torch.device("cuda:0")
graph = dill.load(open(data_dir + 'graph.pk', 'rb'))

In [ ]:
def sample_subgraph(graph, time_range, sampled_depth = 2, sampled_number = 8, inp = None):
    '''
        Sample Sub-Graph based on the connection of other nodes with currently sampled nodes
        We maintain budgets for each node type, indexed by <node_id, time>.
        Currently sampled nodes are stored in layer_data.
        After nodes are sampled, we construct the sampled adjacancy matrix.
    '''
    layer_data  = defaultdict( #target_type
                        lambda: {} # {target_id + time}
                    )
    budget     = defaultdict( #source_type
                                    lambda: defaultdict(  #source_id + source_time
                                        lambda: 0. #sampled_score
                            ))
    new_layer_adj  = defaultdict( #target_type
                                    lambda: defaultdict(  #source_type
                                        lambda: defaultdict(  #relation_type
                                            lambda: [] #[target_id, source_id]
                                )))
    '''
        For each node being sampled, we find out all its neighborhood, 
        adding the degree count of these nodes in the budget.
        Note that there exist some nodes that have many neighborhoods
        (such as fields, venues), for those case, we only consider 
    '''
    def add_budget(te, target_id, target_time, layer_data, budget):
        for source_type in te:
            tes = te[source_type]
            for relation_type in tes:
                if relation_type == 'self':
                    continue
                adl = tes[relation_type][target_id]
                if len(adl) < sampled_number:
                    sampled_ids = list(adl.keys())
                else:
                    sampled_ids = np.random.choice(list(adl.keys()), sampled_number, replace = False)
                for source_id in sampled_ids:
                    source_time = adl[source_id]
                    if source_time == None:
                        source_time = target_time
                    k = encode(source_id, source_time)
                    '''
                        If the node's time is out of range or already being sampled, skip
                        Otherwise, accumulate the normalized degree.
                    '''
                    if source_time not in time_range or k in layer_data[source_type]:
                        continue
                    budget[source_type][k] += 1. / len(sampled_ids)
    '''
        The encode and decode function is used to index each node
        by its node_id and time together. So that a same node with
        different timestamps can exist in the sampled graph.
    '''
    def decode(s):
        idx = s.find('-')
        return np.array([s[:idx], s[idx+1:]], dtype=float)
    def encode(i, t):
        return '%s-%s' % (i, t)

    '''
        If inp == None: we sample some paper as initial nodes;
        else:           we are dealing with a specific supervised task (e.g. author disambiguation),
                        where some node pairs are given as output.
    '''
    
    if inp == None:
        _time = np.random.choice(list(time_range.keys()))
        res = graph.node_feature['paper'][graph.node_feature['paper']['time'] == _time]
        sampn = min(len(res), sampled_number)
        rand_paper_ids  = np.random.choice(list(res.index), sampn, replace = False)
        '''
            First adding the sampled nodes then updating budget.
        '''
        for _id in rand_paper_ids:
            layer_data['paper'][encode(_id, _time)] = len(layer_data['paper'])
        for _id in rand_paper_ids:
            add_budget(graph.edge_list['paper'], _id, _time, layer_data, budget)
    else:
        '''
            First adding the sampled nodes then updating budget.
        '''
        for _type in inp:
            for _id, _time in inp[_type]:
                layer_data[_type][encode(_id, _time)] = len(layer_data[_type])
        for _type in inp:
            te = graph.edge_list[_type]
            for _id, _time in inp[_type]:
                add_budget(te, _id, _time, layer_data, budget)
    '''
        We recursively expand the sampled graph by sampled_depth.
        Each time we sample a fixed number of nodes for each budget,
        based on the accumulated degree.
    '''
    for layer in range(sampled_depth):
        sts = list(budget.keys())
        for source_type in sts:
            te = graph.edge_list[source_type]
            keys  = np.array(list(budget[source_type].keys()))
            if sampled_number > len(keys):
                '''
                    Directly sample all the nodes
                '''
                sampled_ids = np.arange(len(keys))
            else:
                '''
                    Sample based on accumulated degree
                '''
                score = np.array(list(budget[source_type].values())) ** 2
                score = score / np.sum(score)
                sampled_ids = np.random.choice(len(score), sampled_number, p = score, replace = False) 
            sampled_keys = keys[sampled_ids]
            '''
                First adding the sampled nodes then updating budget.
            '''
            for k in sampled_keys:
                layer_data[source_type][k] = len(layer_data[source_type])
            for k in sampled_keys:
                source_id, source_time = decode(k)
                add_budget(te, int(source_id), int(source_time), layer_data, budget)
                budget[source_type].pop(k)    
    '''
        Prepare feature, time and adjacency matrix for the sampled graph
    '''
    feature = {}
    times   = {}
    indxs   = {}
    for _type in layer_data:
        idxs  = np.array([decode(key) for key in layer_data[_type]])
        if 'node_emb' in graph.node_feature[_type]:
            feature[_type] = np.array(list(graph.node_feature['field'].loc[idxs[:,0], 'node_emb']), dtype=np.float)
        else:
            feature[_type] = np.zeros([len(idxs), 400])
        feature[_type] = np.concatenate((feature[_type], list(graph.node_feature[_type].loc[idxs[:,0], 'emb']),\
            np.log10(np.array(list(graph.node_feature[_type].loc[idxs[:,0], 'citation'])).reshape(-1, 1) + 0.01)), axis=1)
        
        times[_type]   = idxs[:,1]
        indxs[_type]   = idxs[:,0]
    edge_list = defaultdict( #target_type
                        lambda: defaultdict(  #source_type
                            lambda: defaultdict(  #relation_type
                                lambda: [] # [target_id, source_id] 
                                    )))
    for _type in layer_data:
        for _key in layer_data[_type]:
            _ser = layer_data[_type][_key]
            edge_list[_type][_type]['self'] += [[_ser, _ser]]
    '''
        Reconstruct sampled adjacancy matrix by checking whether each
        link exist in the original graph
    '''
    for target_type in graph.edge_list:
        te = graph.edge_list[target_type]
        for source_type in te:
            tes = te[source_type]
            for relation_type in tes:
                if relation_type in ['APA_coauthor', 'rev_APV_in', 'IPI_coauthor', 'APV_in']:
                    continue
                tesr = tes[relation_type]
                for target_key in layer_data[target_type]:
                    target_ser = layer_data[target_type][target_key]
                    tesrt = tesr[decode(target_key)[0]]
                    for source_key in layer_data[source_type]:
                        source_ser = layer_data[source_type][source_key]
                        '''
                            Check whether each link (target_id, source_id) exist in original adjacancy matrix
                        '''
                        if decode(source_key)[0] in tesrt:
                            edge_list[target_type][source_type][relation_type] += [[target_ser, source_ser]]
    return feature, times, edge_list, indxs

In [ ]:
# def sample_subgraph(graph, time_range, sampled_depth = 2, sampled_number = 8, inp = None):
#     '''
#         Sample Sub-Graph based on the connection of other nodes with currently sampled nodes
#         We maintain budgets for each node type, indexed by <node_id, time>.
#         Currently sampled nodes are stored in layer_data.
#         After nodes are sampled, we construct the sampled adjacancy matrix.
#     '''
#     layer_data  = defaultdict( #target_type
#                         lambda: {} # {target_id: [ser, time]}
#                     )
#     budget     = defaultdict( #source_type
#                                     lambda: defaultdict(  #source_id
#                                         lambda: [0., 2000] #sampled_score, source_time
#                             ))
#     new_layer_adj  = defaultdict( #target_type
#                                     lambda: defaultdict(  #source_type
#                                         lambda: defaultdict(  #relation_type
#                                             lambda: [] #[target_id, source_id]
#                                 )))
#     '''
#         For each node being sampled, we find out all its neighborhood, 
#         adding the degree count of these nodes in the budget.
#         Note that there exist some nodes that have many neighborhoods
#         (such as fields, venues), for those case, we only consider 
#     '''
#     def add_budget(te, target_id, target_time, layer_data, budget):
#         for source_type in te:
#             tes = te[source_type]
#             for relation_type in tes:
#                 if relation_type == 'self':
#                     continue
#                 adl = tes[relation_type][target_id]
#                 if len(adl) < sampled_number:
#                     sampled_ids = list(adl.keys())
#                 else:
#                     sampled_ids = np.random.choice(list(adl.keys()), sampled_number, replace = False)
#                 for source_id in sampled_ids:
#                     source_time = adl[source_id]
#                     if source_time == None:
#                         source_time = target_time
#                     '''
#                         If the node's time is out of range or already being sampled, skip
#                         Otherwise, accumulate the normalized degree.
#                     '''
#                     if source_time not in time_range or source_id in layer_data[source_type]:
#                         continue
#                     budget[source_type][source_id][0] += 1. / len(sampled_ids)
#                     budget[source_type][source_id][1] = source_time
#     '''
#         The encode and decode function is used to index each node
#         by its node_id and time together. So that a same node with
#         different timestamps can exist in the sampled graph.
#     '''
    
#     if inp == None:
#         _time = np.random.choice(list(time_range.keys()))
#         res = graph.node_feature['paper'][graph.node_feature['paper']['time'] == _time]
#         sampn = min(len(res), sampled_number)
#         rand_paper_ids  = np.random.choice(list(res.index), sampn, replace = False)
#         '''
#             First adding the sampled nodes then updating budget.
#         '''
#         for _id in rand_paper_ids:
#             layer_data['paper'][_id] = [len(layer_data['paper']), _time]
#         for _id in rand_paper_ids:
#             add_budget(graph.edge_list['paper'], _id, _time, layer_data, budget)
#     else:
#         '''
#             First adding the sampled nodes then updating budget.
#         '''
#         for _type in inp:
#             for _id, _time in inp[_type]:
#                 layer_data[_type][_id] = [len(layer_data[_type]), _time]
#         for _type in inp:
#             te = graph.edge_list[_type]
#             for _id, _time in inp[_type]:
#                 add_budget(te, _id, _time, layer_data, budget)
#     '''
#         We recursively expand the sampled graph by sampled_depth.
#         Each time we sample a fixed number of nodes for each budget,
#         based on the accumulated degree.
#     '''
#     for layer in range(sampled_depth):
#         sts = list(budget.keys())
#         for source_type in sts:
#             te = graph.edge_list[source_type]
#             keys  = np.array(list(budget[source_type].keys()))
#             vals  = np.array(list(budget[source_type].values()))
#             if sampled_number > len(keys):
#                 '''
#                     Directly sample all the nodes
#                 '''
#                 sampled_ids = np.arange(len(keys))
#             else:
#                 '''
#                     Sample based on accumulated degree
#                 '''
#                 score = vals[:,0] ** 2
#                 score = score / np.sum(score)
#                 sampled_ids = np.random.choice(len(score), sampled_number, p = score, replace = False) 
#             sampled_keys = keys[sampled_ids]
#             sampled_tims = vals[sampled_ids][:, 1]
#             '''
#                 First adding the sampled nodes then updating budget.
#             '''
#             for k, t in zip(sampled_keys, sampled_tims):
#                 layer_data[source_type][k] = [len(layer_data[source_type]), t]
#             for k, t in zip(sampled_keys, sampled_tims):
#                 add_budget(te, int(k), int(t), layer_data, budget)
#                 budget[source_type].pop(k)    
#     '''
#         Prepare feature, time and adjacency matrix for the sampled graph
#     '''
#     feature = {}
#     times   = {}
#     indxs   = {}
#     for _type in layer_data:
#         idxs = []
#         tims = []
#         for k in layer_data[_type]:
#             idxs += [k]
#             tims += [layer_data[_type][k][1]]
#         if 'node_emb' in graph.node_feature[_type]:
#             feature[_type] = np.array(list(graph.node_feature['field'].loc[idxs, 'node_emb']), dtype=np.float)
#         else:
#             feature[_type] = np.zeros([len(layer_data[_type]), 400])
#         feature[_type] = np.concatenate((feature[_type], list(graph.node_feature[_type].loc[idxs, 'emb']),\
#             np.log10(np.array(list(graph.node_feature[_type].loc[idxs, 'citation'])).reshape(-1, 1) + 0.01)), axis=1)
#         indxs[_type] = idxs
#         times[_type] = tims
        
#     edge_list = defaultdict( #target_type
#                         lambda: defaultdict(  #source_type
#                             lambda: defaultdict(  #relation_type
#                                 lambda: [] # [target_id, source_id] 
#                                     )))
#     for _type in layer_data:
#         for _key in layer_data[_type]:
#             _ser = layer_data[_type][_key][0]
#             edge_list[_type][_type]['self'] += [[_ser, _ser]]
#     '''
#         Reconstruct sampled adjacancy matrix by checking whether each
#         link exist in the original graph
#     '''
#     for target_type in graph.edge_list:
#         te = graph.edge_list[target_type]
#         for source_type in te:
#             tes = te[source_type]
#             for relation_type in tes:
#                 if relation_type in ['APA_coauthor', 'rev_APV_in', 'IPI_coauthor', 'APV_in']:
#                     continue
#                 tesr = tes[relation_type]
#                 for target_key in layer_data[target_type]:
#                     target_ser = layer_data[target_type][target_key][0]
#                     tesrt = tesr[target_key]
#                     for source_key in layer_data[source_type]:
#                         source_ser = layer_data[source_type][source_key][0]
#                         '''
#                             Check whether each link (target_id, source_id) exist in original adjacancy matrix
#                         '''
#                         if source_key in tesrt:
#                             edge_list[target_type][source_type][relation_type] += [[target_ser, source_ser]]
#     return feature, times, edge_list, indxs

In [ ]:
train_range = {t: True for t in graph.times if t != None and t < 2015}
valid_range = {t: True for t in graph.times if t != None and t >= 2015  and t <= 2016}
test_range  = {t: True for t in graph.times if t != None and t > 2016}

In [ ]:
def pf_sample(seed, papers, pairs, t_range, batch_size, test = False):
    np.random.seed(seed)
    _time = np.random.choice(list(papers.keys()))
    sampn = min(len(papers[_time]), batch_size)
    pids = np.array(papers[_time])[np.random.choice(len(papers[_time]), sampn, replace = False)]
    fids = []
    edge = defaultdict(lambda: {})
    for x_id, p_id in enumerate(pids):
        f_ids = pairs[p_id]
        for f_id in f_ids:
            if f_id not in fids:
                fids += [f_id]
            edge[x_id][fids.index(f_id)] = True
    pids = np.stack([pids, np.repeat([_time], sampn)]).T
    fids = np.stack([fids, np.repeat([_time], len(fids))]).T
 
    feature, times, edge_list, _ = sample_subgraph(graph, t_range, \
                inp = {'paper': pids, 'field': fids}, sampled_depth = 3, sampled_number = 100)

    el = []
    for i in edge_list['paper']['field']['rev_PF_in_L2']:
        if i[0] in edge and i[1] in edge[i[0]]:
            continue
        el += [i]
    edge_list['paper']['field']['rev_PF_in_L2'] = el

    el = []
    for i in edge_list['field']['paper']['PF_in_L2']:
        if i[1] in edge and i[0] in edge[i[1]]:
            continue
        el += [i]
    edge_list['field']['paper']['PF_in_L2'] = el
    
    
    node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = \
            to_torch(feature, times, edge_list, graph)
    '''
        Trace the paper_id and field_id by its own index plus the type start index
    '''
    paper_ids = np.arange(len(pids)) + node_dict['paper'][0]
    field_ids = np.arange(len(fids)) + node_dict['field'][0]
    ylabel = torch.zeros(sampn, len(cand_list))
    for x_id, p_id in enumerate(pids[:,0]):
        for f_id in pairs[p_id]:
            ylabel[x_id][cand_list.index(f_id)] = 1
    ylabel /= ylabel.sum(axis=1).view(-1, 1)
    return node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel
    
def prepare_data(pool, process_ids):
    jobs = []
    for process_id in process_ids[:-1]:
        p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), train_papers, \
                                               train_pairs, train_range, batch_size))
        jobs.append(p)
    p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), valid_papers, \
                                           valid_pairs, valid_range, batch_size))
    jobs.append(p)
    return jobs

In [ ]:
class GNN(nn.Module):
    def __init__(self, in_dim, n_hid, num_types, num_relations, n_heads, n_layers, dropout = 0.3):
        super(GNN, self).__init__()
        self.gcs = nn.ModuleList()
        self.num_types = num_types
        self.in_dim    = in_dim
        self.n_hid     = n_hid
        self.aggregat_ws   = nn.ModuleList()
        self.drop          = nn.Dropout(dropout)
        for t in range(num_types):
            self.aggregat_ws.append(nn.Linear(in_dim, n_hid))
        for l in range(n_layers):
            self.gcs.append(RAGCNConv(n_hid, n_hid, num_types, num_relations, n_heads, dropout))
    def set_device(self, device):
        self.device = device
        for gc in self.gcs:
            gc.device = device
    def forward(self, node_feature, node_type, edge_time, edge_index, edge_type):
        res = torch.zeros(node_feature.size(0), self.n_hid).to(node_feature.device)
        for t_id in range(self.num_types):
            aggregat_w = self.aggregat_ws[t_id]
            idx = (node_type == t_id)
            if idx.sum() == 0:
                continue
            res[idx] = torch.tanh(aggregat_w(node_feature[idx]))
        meta_xs = self.drop(res)
        del res
        for gc in self.gcs:
            meta_xs = gc(meta_xs, node_type, edge_index, edge_type, edge_time)
        return meta_xs

In [ ]:
'''
Paper-Field
'''
paper_ser = {}

train_pairs = {}
valid_pairs = {}
test_pairs  = {}

train_papers = {_time: {} for _time in train_range}
valid_papers = {_time: {} for _time in valid_range}
test_papers  = {_time: {} for _time in test_range}

for f_id in graph.edge_list['field']['paper']['PF_in_L2']:
    for p_id in graph.edge_list['field']['paper']['PF_in_L2'][f_id]:
        _time = graph.edge_list['field']['paper']['PF_in_L2'][f_id][p_id]
        if _time in train_range:
            if p_id not in train_pairs:
                train_pairs[p_id] = []
            train_pairs[p_id] += [f_id]
            train_papers[_time][p_id] = True
        elif _time in valid_range:
            if p_id not in valid_pairs:
                valid_pairs[p_id] = []
            valid_pairs[p_id] += [f_id]
            valid_papers[_time][p_id] = True
        else:
            if p_id not in test_pairs:
                test_pairs[p_id] = []
            test_pairs[p_id] += [f_id]
            test_papers[_time][p_id] = True
for _time in list(train_papers.keys()):
    if len(train_papers[_time]) < batch_size // 2:
        train_papers.pop(_time)
    else:
        train_papers[_time] = np.array(list(train_papers[_time].keys()))
for _time in list(valid_papers.keys()):
    if len(valid_papers[_time]) < batch_size // 2:
        valid_papers.pop(_time)
    else:
        valid_papers[_time] = np.array(list(valid_papers[_time].keys()))
for _time in list(test_papers.keys()):
    if len(test_papers[_time]) < batch_size // 2:
        test_papers.pop(_time)
    else:
        test_papers[_time] = np.array(list(test_papers[_time].keys()))

In [ ]:
types = graph.get_types()
cand_list = list(graph.edge_list['field']['paper']['PF_in_L2'])
gnn = GNN(in_dim = len(graph.node_feature['paper']['emb'][0]) + 401, n_hid = 256, num_types = len(types), \
          num_relations = len(graph.get_meta_graph()) + 1, n_heads = 8, n_layers = 3).to(device)
# gnn = torch.load('../pre-train/save/cpc_model.pt').to(device)
classifier = Classifier(256, len(cand_list)).to(device)
model = nn.Sequential(gnn, classifier)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000, eta_min=1e-6)

In [ ]:
stats = []
pool = mp.Pool(4)
process_ids = np.arange(batch_num // 4)
st = time.time()
jobs = prepare_data(pool, process_ids)
train_step = 1500
best_val   = 0
res = []
criterion = nn.KLDivLoss(reduction='batchmean')
for epoch in np.arange(200)+1:
    '''
        Prepare Training and Validation Data
    '''
    train_data = [job.get() for job in jobs[:-1]]
    valid_data = jobs[-1].get()
    pool.close()
    pool.join()
    pool = mp.Pool(4)
    jobs = prepare_data(pool, process_ids)
    et = time.time()
    print('Data Preparation: %.1fs' % (et - st))
    
    model.train()
    train_losses = []
    torch.cuda.empty_cache()
    for batch in np.arange(2):
        for node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel in train_data:
            node_rep = gnn.forward(node_feature.to(device), node_type.to(device), \
                                   edge_time.to(device), edge_index.to(device), edge_type.to(device))
            res  = classifier.forward(node_rep[paper_ids])
            loss = criterion(res, ylabel.to(device))
            optimizer.zero_grad() 
            torch.cuda.empty_cache()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.2)
            optimizer.step()
            train_losses += [loss.cpu().detach().tolist()]
            train_step += 1
            scheduler.step(train_step)
            del res, loss
    '''
        Valid
    '''
    model.eval()
    with torch.no_grad():
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = valid_data
        node_rep = gnn.forward(node_feature.to(device), node_type.to(device), \
                                   edge_time.to(device), edge_index.to(device), edge_type.to(device))
        res  = classifier.forward(node_rep[paper_ids])
        loss = criterion(res, ylabel.to(device))
        valid_res = []

        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            valid_res += [ai[bi].tolist()]
        valid_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in valid_res])
        if valid_ndcg > best_val:
            best_val = valid_ndcg
            torch.save(model, './save/rgt_1.pt')
        st = time.time()
        print(("Epoch: %d (%.1fs)  LR: %.5f Train Loss: %.2f  Valid Loss: %.2f  Valid NDCG: %.4f") % \
              (epoch, (st-et), optimizer.param_groups[0]['lr'], np.average(train_losses), loss.cpu().detach().tolist(),\
              valid_ndcg))
        stats += [[np.average(train_losses), loss.cpu().detach().tolist()]]
        del res, loss
        if epoch % 5 == 0:
            '''
                Test
            '''
            _time = np.random.choice(list(test_papers.keys()))
            node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                           test_pairs, test_range, batch_size, test=True)
            paper_rep = gnn.forward(node_feature.to(device), node_type.to(device), \
                      edge_time.to(device), edge_index.to(device), edge_type.to(device))[paper_ids]
            res  = classifier.forward(paper_rep)
            test_res = []
            for ai, bi in zip(ylabel, res.argsort(descending = True)):
                test_res += [ai[bi].tolist()]
            test_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in test_res])
            print(test_ndcg)
            del res
    del train_data, valid_data

In [ ]:
stats = np.array(stats)
plt.plot(stats[:,0])
plt.plot(stats[:,1])
plt.show()

In [ ]:
model.eval()
gnn, classifier = model
with torch.no_grad():
    test_res = []
    for _ in range(10):
        _time = np.random.choice(list(test_papers.keys()))
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                       test_pairs, test_range, batch_size, test=True)
        paper_rep = gnn.forward(node_feature.to(device), node_type.to(device), \
                      edge_time.to(device), edge_index.to(device), edge_type.to(device))[paper_ids]
        res = classifier.forward(paper_rep)
        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            test_res += [ai[bi].tolist()]
    test_ndcg = [ndcg_at_k(resi, len(resi)) for resi in test_res]
    print(np.average(test_ndcg), np.var(test_ndcg))
    test_mrr = mean_reciprocal_rank(test_res)
    print(np.average(test_mrr), np.var(test_mrr))

In [ ]:
best_model = torch.load('./save/rgt_1.pt')

In [ ]:
best_model.eval()
gnn, classifier = best_model
with torch.no_grad():
    test_res = []
    for _ in range(10):
        _time = np.random.choice(list(test_papers.keys()))
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                       test_pairs, test_range, batch_size, test=True)
        paper_rep = gnn.forward(node_feature.to(device), node_type.to(device), \
                      edge_time.to(device), edge_index.to(device), edge_type.to(device))[paper_ids]
        res = classifier.forward(paper_rep)
        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            test_res += [ai[bi].tolist()]
    test_ndcg = [ndcg_at_k(resi, len(resi)) for resi in test_res]
    print(np.average(test_ndcg), np.var(test_ndcg))
    test_mrr = mean_reciprocal_rank(test_res)
    print(np.average(test_mrr), np.var(test_mrr))